# Example (bash) commands for preprocessing and Python code for hybrid modeling

## Generating an MSA with Jackhmmer

Jackhmmer, implemented in the HMMER suite, is available at http://hmmer.org/download.html.
To perform a Jackhmmer search with a query sequence, e.g., `pabp_yeast.fasta`, against a locally stored database, e.g., `uniref100.fasta` (available at https://ftp.uniprot.org/pub/databases/uniprot/uniref/uniref100/ but not provided here as the file size is very large), using a bit score of 0.5 (incT = 75 * 0.5 = 37.5), run:

## Post-processing of the MSA
To trim the MSA, excluding all gap positions of the wild-type, positions with more than 30 % gaps, and sequences with more than 50 % gaps, run the provided Python script (input alignment format: Stockholm (.sto), output alignment format: A2M (.a2m)):

## Computing evolutionary local and coupling terms with PLMC
To derive evolutionary terms from the trimmed MSA, which are needed to generate sequence encodings for training supervised models and for statistical modeling, PLMC is used, which is available at https://github.com/debbiemarkslab/plmc. Here, to run PLMC, `-le` is set as previously output/printed by the `sto2a2m.py` script (`-le`= 0.2·(N_sites − 1) and `-lh`=0.01 by default), and the maximum number of iterations (`-m`) is set to 3500 (flag `-n`/`--ncores` only available when compiled with OpenMP):


## Encoding sequences
The output file (`pabp_yeast_plmc.params`), wich contains the coupling parameters, can then be used to encode sequences for supervised modeling by running the provided Python script `make_dataframe.py`. The script generates features for the sequences provided in the input CSV file (the fitness column is labeled with `y`) and writes the features to a new "encoded CSV" file; remember to shift the starting position according to the wild type sequence position of the first position in the trimmed MSA:

## Generating and testing a hybrid model
To generate a hybrid model, we use the `Hybrid_Model` class of the Python script `hybrid_model.py` and load the wild-type sequence (`PABP_YEAST_Fields2013-singles_wt_encoded.npy`) and the encoded CSV file (`PABP_YEAST_Fields2013-singles_encoded.csv`). Then, the "PABP `Hybrid_Model` class instance" can be used to compute the performance of the hybrid model using 80% of the data for training and 20% for testing (`performance_80_20=hybrid_model.performance()`). Noteworthy, the statistical DCA model is generated in the background and no command has to be specified explicitly. Now we can compute the performance of the hybrid model by correlating the predictions and the measured (true) fitness values of the (left-out) test data. As performance metric we use the Spearman rank correlation coefficient:

In [1]:
import os
import numpy as np
from scripts.hybrid_model import Hybrid_Model

hybrid_model=Hybrid_Model(
    os.path.join('pabp_files', 'PABP_YEAST_Fields2013-singles_wt_encoded.npy'),
    os.path.join('pabp_files', 'PABP_YEAST_Fields2013-singles_encoded.csv')
)

performance_80_20=hybrid_model.performance()
X_singles,y_singles=hybrid_model.X,hybrid_model.y  # saving all single-substution encoding-fitness data in memory for later on

print('Spearmanr singles 80/20: %.2f +/- %.2f'%(performance_80_20.mean(), performance_80_20.std(ddof=1)))

Spearmanr singles 80/20: 0.73 +/- 0.05


# Training on single substituted variant data and predicting recombinants
We can also use the hybrid model instance to optimize the individual model contributions `beta1` (statistical DCA model) and `beta2` (linear model `ridge` trained in a supervised manner) on all single substituted variant data and predict all recombinants that are in the encoded test set, i.e., in `PABP_YEAST_Fields2013-doubles_encoded.csv`, to estimate the generalization performance for predicting the fitness of recombinant variants:

In [2]:
import pandas as pd
from scipy.stats import spearmanr

df=pd.read_csv(os.path.join('pabp_files','PABP_YEAST_Fields2013-doubles_encoded.csv'),sep=';')
X_doubles=df.iloc[:,2:].to_numpy()
y_doubles=df.iloc[:,1].to_numpy()

# Using all single-substution encoding-fitness data to construct a new hybrid model
beta1,beta2,ridge=hybrid_model._settings(X_singles,y_singles)  # Getting individual model contributions and the trained RidgeR.
y_doubles_pred=hybrid_model.predict(X_doubles, ridge, beta1, beta2)  # Predict all recombinants of entire doubles set

print('Spearmanr doubles 80/20: %.2f'%(spearmanr(y_doubles, y_doubles_pred)[0]))

Spearmanr doubles 80/20: 0.78


Done!